In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Objectif du projet de ce projet est de faire un classement avec les équipe qui ont plus marqués cette saison et à la fin ,en faire un classement


In [31]:
!pip install pyspark

In [25]:
#Impory necessary libtairies
from pyspark.sql import SparkSession,Window
from pyspark.sql.functions import *
from pyspark.sql.functions import col, rank


In [6]:
#Create a park session
spark = SparkSession.builder.appName("DataAnalysis").getOrCreate()

In [7]:
#Load dataset
df_matches = spark.read.format('csv').options(header = 'True').load('/content/drive/MyDrive/Colab Notebooks/Data/football_matches.csv')
#Preview of data

df_matches.limit(20).show()

+--------+---+------+----------+------------------+--------------+----+----+---+
|Match_ID|Div|Season|      Date|          HomeTeam|      AwayTeam|FTHG|FTAG|FTR|
+--------+---+------+----------+------------------+--------------+----+----+---+
|       1| D2|  2009|2010-04-04|        Oberhausen|Kaiserslautern|   2|   1|  H|
|       2| D2|  2009|2009-11-01|       Munich 1860|Kaiserslautern|   0|   1|  A|
|       3| D2|  2009|2009-10-04|     Frankfurt FSV|Kaiserslautern|   1|   1|  D|
|       4| D2|  2009|2010-02-21|     Frankfurt FSV|     Karlsruhe|   2|   1|  H|
|       5| D2|  2009|2009-12-06|             Ahlen|     Karlsruhe|   1|   3|  A|
|       6| D2|  2009|2010-04-03|      Union Berlin|     Karlsruhe|   1|   1|  D|
|       7| D2|  2009|2009-08-14|         Paderborn|     Karlsruhe|   2|   0|  H|
|       8| D2|  2009|2010-03-08|         Bielefeld|     Karlsruhe|   0|   1|  A|
|       9| D2|  2009|2009-09-26|    Kaiserslautern|     Karlsruhe|   2|   0|  H|
|      10| D2|  2009|2009-11

Meaning of the features:

*   Match_ID : Unique identifieer for each match
*   Div : Championship level (D1 = first level,D2 = Second level,E0 =third level)


*   Season :Starting year of the season in wich the match took place
* Date : Date of the match
*Home Team :Team playing  in the stadium
* Away Team : Team lplaying in the opposong team's stadium
*FTHG :Number of goals of Home Team
*FTAG :Number of goals of Away Team
*FTR : Final Result





In [8]:
df_matches = df_matches.selectExpr("*", "`FTHG` AS `HomeTeamGoals`",

                                   "`FTAG` AS `AwayTeamGoals`",

                                   "`FTR` AS `FinalResult`"

                                   )

df_matches.limit(20).show()

+--------+---+------+----------+------------------+--------------+----+----+---+-------------+-------------+-----------+
|Match_ID|Div|Season|      Date|          HomeTeam|      AwayTeam|FTHG|FTAG|FTR|HomeTeamGoals|AwayTeamGoals|FinalResult|
+--------+---+------+----------+------------------+--------------+----+----+---+-------------+-------------+-----------+
|       1| D2|  2009|2010-04-04|        Oberhausen|Kaiserslautern|   2|   1|  H|            2|            1|          H|
|       2| D2|  2009|2009-11-01|       Munich 1860|Kaiserslautern|   0|   1|  A|            0|            1|          A|
|       3| D2|  2009|2009-10-04|     Frankfurt FSV|Kaiserslautern|   1|   1|  D|            1|            1|          D|
|       4| D2|  2009|2010-02-21|     Frankfurt FSV|     Karlsruhe|   2|   1|  H|            2|            1|          H|
|       5| D2|  2009|2009-12-06|             Ahlen|     Karlsruhe|   1|   3|  A|            1|            3|          A|
|       6| D2|  2009|2010-04-03|

In [9]:
#drop columns FTHG FTAG FTR
df_matches = df_matches.drop("FTHG","FTAG","FTR")
#vIEW DATAS
df_matches.limit(5).show()

+--------+---+------+----------+-------------+--------------+-------------+-------------+-----------+
|Match_ID|Div|Season|      Date|     HomeTeam|      AwayTeam|HomeTeamGoals|AwayTeamGoals|FinalResult|
+--------+---+------+----------+-------------+--------------+-------------+-------------+-----------+
|       1| D2|  2009|2010-04-04|   Oberhausen|Kaiserslautern|            2|            1|          H|
|       2| D2|  2009|2009-11-01|  Munich 1860|Kaiserslautern|            0|            1|          A|
|       3| D2|  2009|2009-10-04|Frankfurt FSV|Kaiserslautern|            1|            1|          D|
|       4| D2|  2009|2010-02-21|Frankfurt FSV|     Karlsruhe|            2|            1|          H|
|       5| D2|  2009|2009-12-06|        Ahlen|     Karlsruhe|            1|            3|          A|
+--------+---+------+----------+-------------+--------------+-------------+-------------+-----------+



In [10]:
df_matches.toPandas().head(5)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,FinalResult
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


In [11]:
#Create binary columns for win losses or draw

df_matches = df_matches.withColumn("HomeTeamwin",when(col("FinalResult") == "H",1).otherwise(0))\
.withColumn("AwayTeamwin",when(col("FinalResult")== "A",1).otherwise(0))\
.withColumn("GameTie",when(col("FinalResult")== "D",1).otherwise(0))

#vIEW DATAS
df_matches.limit(5).show()
from pyspark.sql import functions as F

# Assuming df_matches is your PySpark DataFrame
#df_matches = df_matches.withColumn("HomeTeamwin", F.when(F.col("FinalResult") == "H", 1).otherwise(0))\.withColumn

# View data
#df_matches.limit(5).show()

+--------+---+------+----------+-------------+--------------+-------------+-------------+-----------+-----------+-----------+-------+
|Match_ID|Div|Season|      Date|     HomeTeam|      AwayTeam|HomeTeamGoals|AwayTeamGoals|FinalResult|HomeTeamwin|AwayTeamwin|GameTie|
+--------+---+------+----------+-------------+--------------+-------------+-------------+-----------+-----------+-----------+-------+
|       1| D2|  2009|2010-04-04|   Oberhausen|Kaiserslautern|            2|            1|          H|          1|          0|      0|
|       2| D2|  2009|2009-11-01|  Munich 1860|Kaiserslautern|            0|            1|          A|          0|          1|      0|
|       3| D2|  2009|2009-10-04|Frankfurt FSV|Kaiserslautern|            1|            1|          D|          0|          0|      1|
|       4| D2|  2009|2010-02-21|Frankfurt FSV|     Karlsruhe|            2|            1|          H|          1|          0|      0|
|       5| D2|  2009|2009-12-06|        Ahlen|     Karlsruhe| 

In [12]:
#Create a DataFrame Containing only Bundesliga matches
df_bundesliga = df_matches.filter((col('Div')=='D1')&(col('Season')>=2000)&(col('Season')<=2015))
#df_bundesliga.limit(100).show()
#vIEW THE DATA
df_bundesliga.toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,FinalResult,HomeTeamwin,AwayTeamwin,GameTie
0,21,D1,2009,2010-02-06,Bochum,Leverkusen,1,1,D,0,0,1
1,22,D1,2009,2009-11-22,Bayern Munich,Leverkusen,1,1,D,0,0,1
2,23,D1,2009,2010-05-08,M'gladbach,Leverkusen,1,1,D,0,0,1
3,24,D1,2009,2009-08-08,Mainz,Leverkusen,2,2,D,0,0,1
4,25,D1,2009,2009-10-17,Hamburg,Leverkusen,0,0,D,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4891,37179,D1,2004,2005-05-21,Hertha,Hannover,0,0,D,0,0,1
4892,37180,D1,2004,2005-05-21,Kaiserslautern,Werder Bremen,1,2,A,0,1,0
4893,37181,D1,2004,2005-05-21,Leverkusen,M'gladbach,5,1,H,1,0,0
4894,37182,D1,2004,2005-05-21,Nurnberg,Mainz,1,2,A,0,1,0


In [13]:
#Create a DataFrame containing statistics on home matches
#df_home_matches = df_bundesliga.groupeby('Season','HomeTeam') \ .agg(sum('HomeTeamwin').alias('TotalHomewin'),
      #sum('AwayTeamwin').alias('TotalHomeLoss'),
      # sum('GameTie').alias('TotalHomeTie'),
     #sum('HomeTeamGoals').alias('HomeScoreGoals'),
      #sum('AwayTeamGoals').alias('HomeagainstGoals'))

df_home_matches = df_bundesliga.groupby('Season', 'HomeTeam') \
    .agg(
        sum('HomeTeamwin').alias('TotalHomewin'),
        sum('AwayTeamwin').alias('TotalHomeLoss'),
        sum('GameTie').alias('TotalHomeTie'),
        sum('HomeTeamGoals').alias('HomeScoreGoals'),
        sum('AwayTeamGoals').alias('HomeagainstGoals')
    ) \
    .withColumnRenamed('HomeTeam', 'Team')

In [14]:
#df_home_matches.limit(100).show()
df_home_matches.toPandas()

,Season,Team,TotalHomewin,TotalHomeLoss,TotalHomeTie,HomeScoreGoals,HomeagainstGoals
0,2011,Hamburg,3,7,7,19.0,29.0
1,2005,Kaiserslautern,5,7,5,26.0,33.0
2,2006,Cottbus,6,6,5,21.0,22.0
3,2001,St Pauli,4,9,4,19.0,28.0
4,2005,Mainz,6,4,7,31.0,23.0
...,...,...,...,...,...,...,...
283,2004,Bielefeld,7,7,3,21.0,21.0
284,2004,Werder Bremen,9,4,4,33.0,15.0
285,2013,Augsburg,9,5,3,27.0,22.0
286,2004,Stuttgart,12,3,2,34.0,15.0


In [15]:
#Create a DataFrame containing statistics on away matches
df_away_matches = df_bundesliga.groupby('Season', 'AwayTeam') \
    .agg(
        sum('AwayTeamwin').alias('TotalAwaywin'),
        sum('HomeTeamwin').alias('TotalAwayLoss'),
        sum('GameTie').alias('TotalAwayTie'),
        sum('AwayTeamGoals').alias('AwayScoreGoals'),
        sum('HomeTeamGoals').alias('AwayagainstGoals')
    ) \
    .withColumnRenamed('AwayTeam', 'Team')

In [16]:
#df_home_matches.limit(100).show()
df_away_matches.toPandas()

,Season,Team,TotalAwaywin,TotalAwayLoss,TotalAwayTie,AwayScoreGoals,AwayagainstGoals
0,2011,Hamburg,5,7,5,16.0,28.0
1,2005,Kaiserslautern,3,10,4,21.0,38.0
2,2006,Cottbus,5,9,3,17.0,27.0
3,2001,St Pauli,0,11,6,18.0,42.0
4,2005,Mainz,3,10,4,15.0,24.0
...,...,...,...,...,...,...,...
283,2004,Bielefeld,4,9,4,16.0,28.0
284,2004,Werder Bremen,9,7,1,35.0,22.0
285,2013,Augsburg,6,7,4,20.0,25.0
286,2004,Stuttgart,5,7,5,20.0,25.0


In [17]:
#Join Home and Away DATA
df_merged = df_home_matches.join(df_away_matches, ['Team', 'Season'], 'inner')

# Converting the merged PySpark DataFrame to a Pandas DataFrame
df_merged.toPandas()

,Team,Season,TotalHomewin,TotalHomeLoss,TotalHomeTie,HomeScoreGoals,HomeagainstGoals,TotalAwaywin,TotalAwayLoss,TotalAwayTie,AwayScoreGoals,AwayagainstGoals
0,Hamburg,2011,3,7,7,19.0,29.0,5,7,5,16.0,28.0
1,Kaiserslautern,2005,5,7,5,26.0,33.0,3,10,4,21.0,38.0
2,Cottbus,2006,6,6,5,21.0,22.0,5,9,3,17.0,27.0
3,St Pauli,2001,4,9,4,19.0,28.0,0,11,6,18.0,42.0
4,Mainz,2005,6,4,7,31.0,23.0,3,10,4,15.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...
283,Bielefeld,2004,7,7,3,21.0,21.0,4,9,4,16.0,28.0
284,Werder Bremen,2004,9,4,4,33.0,15.0,9,7,1,35.0,22.0
285,Augsburg,2013,9,5,3,27.0,22.0,6,7,4,20.0,25.0
286,Stuttgart,2004,12,3,2,34.0,15.0,5,7,5,20.0,25.0


In [18]:
# Creating new columns for total scores, total goals against, total wins, total losses, and total ties
df_totals = df_merged.withColumn('GoalScored', col('HomeScoreGoals') + col('AwayScoreGoals')) \
                    .withColumn('GoalsAgainst', col('HomeagainstGoals') + col('AwayagainstGoals')) \
                    .withColumn('Win', col('TotalHomewin') + col('TotalAwaywin')) \
                    .withColumn('Loss', col('TotalHomeLoss') + col('TotalAwayLoss')) \
                    .withColumn('Tie', col('TotalHomeTie') + col('TotalAwayTie'))

In [19]:
# Converting the merged PySpark DataFrame to a Pandas DataFrame
df_totals.toPandas()

,Team,Season,TotalHomewin,TotalHomeLoss,TotalHomeTie,HomeScoreGoals,HomeagainstGoals,TotalAwaywin,TotalAwayLoss,TotalAwayTie,AwayScoreGoals,AwayagainstGoals,GoalScored,GoalsAgainst,Win,Loss,Tie
0,Hamburg,2011,3,7,7,19.0,29.0,5,7,5,16.0,28.0,35.0,57.0,8,14,12
1,Kaiserslautern,2005,5,7,5,26.0,33.0,3,10,4,21.0,38.0,47.0,71.0,8,17,9
2,Cottbus,2006,6,6,5,21.0,22.0,5,9,3,17.0,27.0,38.0,49.0,11,15,8
3,St Pauli,2001,4,9,4,19.0,28.0,0,11,6,18.0,42.0,37.0,70.0,4,20,10
4,Mainz,2005,6,4,7,31.0,23.0,3,10,4,15.0,24.0,46.0,47.0,9,14,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,Bielefeld,2004,7,7,3,21.0,21.0,4,9,4,16.0,28.0,37.0,49.0,11,16,7
284,Werder Bremen,2004,9,4,4,33.0,15.0,9,7,1,35.0,22.0,68.0,37.0,18,11,5
285,Augsburg,2013,9,5,3,27.0,22.0,6,7,4,20.0,25.0,47.0,47.0,15,12,7
286,Stuttgart,2004,12,3,2,34.0,15.0,5,7,5,20.0,25.0,54.0,40.0,17,10,7


In [20]:
#Unnecessary colums
cols_to_drop = ['TotalHomewin	','TotalHomeLoss',	'TotalHomeTie',	'HomeScoreGoals',	'HomeagainstGoals',	'TotalAwaywin'	,'TotalAwayLoss',	'TotalAwayTie	','AwayScoreGoals',	'AwayagainstGoals']
#Drop Unnecessary colums
df_cleaned = df_totals.drop(*cols_to_drop)

In [21]:
df_cleaned.toPandas()

,Team,Season,TotalHomewin,TotalAwayTie,GoalScored,GoalsAgainst,Win,Loss,Tie
0,Hamburg,2011,3,5,35.0,57.0,8,14,12
1,Kaiserslautern,2005,5,4,47.0,71.0,8,17,9
2,Cottbus,2006,6,3,38.0,49.0,11,15,8
3,St Pauli,2001,4,6,37.0,70.0,4,20,10
4,Mainz,2005,6,4,46.0,47.0,9,14,11
...,...,...,...,...,...,...,...,...,...
283,Bielefeld,2004,7,4,37.0,49.0,11,16,7
284,Werder Bremen,2004,9,1,68.0,37.0,18,11,5
285,Augsburg,2013,9,4,47.0,47.0,15,12,7
286,Stuttgart,2004,12,5,54.0,40.0,17,10,7


In [22]:
#Create additional colums
df_processed = df_cleaned.withColumn('GoalDifferentials', col('GoalScored') - col('GoalsAgainst')) \
                         .withColumn('WinPercentage', round(100 * col('Win') / (col('Win') + col('Loss') + col('Tie')),2))
df_processed.toPandas()

,Team,Season,TotalHomewin,TotalAwayTie,GoalScored,GoalsAgainst,Win,Loss,Tie,GoalDifferentials,WinPercentage
0,Hamburg,2011,3,5,35.0,57.0,8,14,12,-22.0,23.53
1,Kaiserslautern,2005,5,4,47.0,71.0,8,17,9,-24.0,23.53
2,Cottbus,2006,6,3,38.0,49.0,11,15,8,-11.0,32.35
3,St Pauli,2001,4,6,37.0,70.0,4,20,10,-33.0,11.76
4,Mainz,2005,6,4,46.0,47.0,9,14,11,-1.0,26.47
...,...,...,...,...,...,...,...,...,...,...,...
283,Bielefeld,2004,7,4,37.0,49.0,11,16,7,-12.0,32.35
284,Werder Bremen,2004,9,1,68.0,37.0,18,11,5,31.0,52.94
285,Augsburg,2013,9,4,47.0,47.0,15,12,7,0.0,44.12
286,Stuttgart,2004,12,5,54.0,40.0,17,10,7,14.0,50.00


In [28]:
#SET WINDOW PARTITION
from pyspark.sql import Window
from pyspark.sql.functions import col, rank

# Définition de la fenêtre de partition
window_partition = Window.partitionBy('Season').orderBy(col('Winpercentage').desc(), col('GoalDifferentials').desc())

# Classement des équipes par saison
df_ranked = df_processed.withColumn("TeamPosition", rank().over(window_partition))

df_ranked.toPandas()

,Team,Season,TotalHomewin,TotalAwayTie,GoalScored,GoalsAgainst,Win,Loss,Tie,GoalDifferentials,WinPercentage,TeamPosition
0,Bayern Munich,2000,12,5,62.0,37.0,19,9,6,25.0,55.88,1
1,Schalke 04,2000,12,4,65.0,35.0,18,8,8,30.0,52.94,2
2,Hertha,2000,11,0,58.0,52.0,18,14,2,6.0,52.94,3
3,Leverkusen,2000,10,4,54.0,40.0,17,11,6,14.0,50.00,4
4,Dortmund,2000,9,6,62.0,42.0,16,8,10,20.0,47.06,5
...,...,...,...,...,...,...,...,...,...,...,...,...
283,Hoffenheim,2015,6,4,39.0,54.0,9,15,10,-15.0,26.47,14
284,Darmstadt,2015,2,5,38.0,53.0,9,14,11,-15.0,26.47,14
285,Ein Frankfurt,2015,6,3,34.0,52.0,9,16,9,-18.0,26.47,16
286,Stuttgart,2015,6,5,50.0,75.0,9,19,6,-25.0,26.47,17


In [30]:
#Filtrer les teams
df_top_teams = df_ranked.filter(col('TeamPosition')==1)

#Preview of the data

df_top_teams.toPandas()
#les meilleurs équipes de la BundesLiga de 2000 à 2015



,Team,Season,TotalHomewin,TotalAwayTie,GoalScored,GoalsAgainst,Win,Loss,Tie,GoalDifferentials,WinPercentage,TeamPosition
0,Bayern Munich,2000,12,5,62.0,37.0,19,9,6,25.0,55.88,1
1,Leverkusen,2001,14,5,77.0,38.0,21,7,6,39.0,61.76,1
2,Bayern Munich,2002,13,3,70.0,25.0,23,5,6,45.0,67.65,1
3,Werder Bremen,2003,11,4,79.0,38.0,22,4,8,41.0,64.71,1
4,Bayern Munich,2004,14,3,75.0,33.0,24,5,5,42.0,70.59,1
5,Bayern Munich,2005,14,7,67.0,32.0,22,3,9,35.0,64.71,1
6,Stuttgart,2006,12,4,61.0,37.0,21,6,7,24.0,61.76,1
7,Bayern Munich,2007,12,5,68.0,21.0,22,2,10,47.0,64.71,1
8,Wolfsburg,2008,16,5,80.0,41.0,21,7,6,39.0,61.76,1
9,Bayern Munich,2009,12,6,72.0,31.0,20,4,10,41.0,58.82,1
